In [2]:
import logging

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

In [3]:
items = pd.read_parquet("items.par")
events = pd.read_parquet("events.par")
items.rename(columns={'book_id': 'item_id'}, inplace=True)
als_recommendations = pd.read_parquet("als_recommendations.parquet")


In [4]:
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2017-08-01").date()

train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx] # ваш код здесь #
events_test = events[~train_test_global_time_split_idx]

# количество пользователей в train и test
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates()# ваш код здесь #
# количество пользователей, которые есть и в train, и в test
common_users =  set(users_train) & set(users_test)# ваш код здесь #

print(len(users_train), len(users_test), len(common_users))

428220 123223 120858


In [5]:
import scipy
import sklearn.preprocessing

# перекодируем идентификаторы пользователей: 
# из имеющихся в последовательность 0, 1, 2, ...
user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(events["user_id"])
events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])

# перекодируем идентификаторы объектов: 
# из имеющихся в последовательность 0, 1, 2, ...
item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(items["item_id"])
items["item_id_enc"] = item_encoder.transform(items["item_id"])
events_train["item_id_enc"] = item_encoder.transform(events_train["item_id"])
events_test["item_id_enc"] = item_encoder.transform(events_test["item_id"])
srt = sorted(events_train['item_id_enc'], reverse=True)
srt[:5]

/tmp/ipykernel_25122/782784623.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
/tmp/ipykernel_25122/782784623.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])
/tmp/ipykernel_25122/782784623.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

[43304, 43304, 43250, 43232, 43232]

In [7]:
# создаём sparse-матрицу формата CSR 
user_item_matrix_train = scipy.sparse.csr_matrix((
    events_train["rating"],
    (events_train['user_id_enc'], events_train['item_id_enc'])),
    dtype=np.int8)

In [8]:
from implicit.als import AlternatingLeastSquares

als_model = AlternatingLeastSquares(factors=50, iterations=50, regularization=0.05, random_state=0)
als_model.fit(user_item_matrix_train)

/home/mle-user/mle_projects/mle-recsys-start/env_recsys_start/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mle-user/mle_projects/mle-recsys-start/env_recsys_start/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 50/50 [03:02<00:00,  3.66s/it]


In [14]:
# получим энкодированные идентификаторы всех объектов, известных нам из events_train
train_item_ids_enc = events_train['item_id_enc'].unique()

max_similar_items = 10

# получаем списки похожих объектов, используя ранее полученную ALS-модель
# метод similar_items возвращает и сам объект, как наиболее похожий
# этот объект мы позже отфильтруем, но сейчас запросим на 1 больше
similar_items = als_model.similar_items(train_item_ids_enc, N=max_similar_items+1)

# преобразуем полученные списки в табличный формат
sim_item_item_ids_enc = similar_items[0]
sim_item_scores = similar_items[1]

similar_items = pd.DataFrame({
    "item_id_enc": train_item_ids_enc,
    "sim_item_id_enc": sim_item_item_ids_enc.tolist(), 
    "score": sim_item_scores.tolist()
    })
similar_items = similar_items.explode(["sim_item_id_enc", "score"], ignore_index=True)

# приводим типы данных
similar_items["sim_item_id_enc"] = similar_items["sim_item_id_enc"].astype("int")
similar_items["score"] = similar_items["score"].astype("float")

# получаем изначальные идентификаторы
similar_items["item_id_1"] = item_encoder.inverse_transform(similar_items["item_id_enc"])
similar_items["item_id_2"] = item_encoder.inverse_transform(similar_items["sim_item_id_enc"])

similar_items = similar_items.drop(columns=["item_id_enc", "sim_item_id_enc"])

# убираем пары с одинаковыми объектами
similar_items = similar_items.query("item_id_1 != item_id_2")

In [21]:
similar_items[similar_items['item_id_1'] == 7126]

,score,item_id_1,item_id_2
10440,0.948725,7126,7190
10441,0.940997,7126,24280
10442,0.930144,7126,1953
10443,0.925066,7126,58696
10444,0.916340,7126,38296
10445,0.916015,7126,2932
10446,0.913951,7126,7184
10447,0.911433,7126,387749
10448,0.909872,7126,7733
10449,0.909454,7126,30597


In [22]:
similar_items.to_parquet("similar_items.parquet")

In [23]:
def print_sim_items(item_id, similar_items):

    item_columns_to_use = ["item_id", "author", "title", "genre_and_votes", "average_rating", "ratings_count"]
    
    item_id_1 = items.query("item_id == @item_id")[item_columns_to_use]
    display(item_id_1)
    
    si = similar_items.query("item_id_1 == @item_id")
    si = si.merge(items[item_columns_to_use].set_index("item_id"), left_on="item_id_2", right_index=True)
    display(si)

In [25]:
print_sim_items(17245, similar_items)

,item_id,author,title,genre_and_votes,average_rating,ratings_count
1058909,17245,"Bram Stoker, Nina Auerbach, David J. Skal",Dracula,"{'Classics': 19603, 'Horror': 10601, 'Fiction'...",3.98,636895


,score,item_id_1,item_id_2,author,title,genre_and_votes,average_rating,ratings_count
24278,0.928823,17245,480204,"Gaston Leroux, Alexander Teixeira de Mattos",The Phantom of the Opera,"{'Classics': 7010, 'Fiction': 2103, 'Horror': ...",3.97,144859
24279,0.900337,17245,51496,"Robert Louis Stevenson, Vladimir Nabokov, Merv...",The Strange Case of Dr. Jekyll and Mr. Hyde,"{'Classics': 12342, 'Fiction': 4037, 'Horror':...",3.79,229898
24280,0.898938,17245,93261,Washington Irving,The Legend of Sleepy Hollow,"{'Classics': 2594, 'Horror': 1182, 'Fiction': ...",3.74,26776
24281,0.897706,17245,295,Robert Louis Stevenson,Treasure Island,"{'Classics': 11249, 'Fiction': 4405, 'Adventur...",3.82,274424
24282,0.896470,17245,2623,"Charles Dickens, Marisa Sestino",Great Expectations,"{'Classics': 19645, 'Fiction': 6662, 'Literatu...",3.75,468462
24283,0.895993,17245,18254,"Charles Dickens, Philip Horne, Gerald Dickens",Oliver Twist,"{'Classics': 11450, 'Fiction': 3656, 'Historic...",3.85,235560
24284,0.886899,17245,7190,Alexandre Dumas,"The Three Musketeers (The D'Artagnan Romances,...","{'Classics': 9823, 'Fiction': 3256, 'Historica...",4.06,198892
24285,0.881911,17245,24213,"Lewis Carroll, John Tenniel, Martin Gardner",Alice's Adventures in Wonderland & Through the...,"{'Classics': 11568, 'Fantasy': 6184, 'Fiction'...",4.06,344482
24286,0.878392,17245,2932,"Daniel Defoe, Virginia Woolf",Robinson Crusoe,"{'Classics': 7725, 'Fiction': 3305, 'Adventure...",3.66,181415
24287,0.870232,17245,1953,"Charles Dickens, Richard Maxwell",A Tale of Two Cities,"{'Classics': 20021, 'Fiction': 6969, 'Historic...",3.82,646983


In [31]:
i2i = similar_items[similar_items['item_id_1'] == 17245].head(3)
i2i = i2i[["item_id_2", "score"]].to_dict(orient="list")
i2i

{'item_id_2': [480204, 51496, 93261],
 'score': [0.9288230538368225, 0.9003370404243469, 0.8989384174346924]}

In [4]:
events

,user_id,item_id,started_at,read_at,is_read,rating,is_reviewed
6679625,1000000,11012,2015-12-05,2015-12-11,True,4,False
6679617,1000000,4671,2014-06-05,2014-06-30,True,5,False
6679618,1000000,5,2012-10-02,2012-10-24,True,5,False
6679620,1000000,2,2009-07-12,2009-07-29,True,5,False
6679621,1000000,14497,2016-05-09,2016-06-02,True,5,False
...,...,...,...,...,...,...,...
5625379,1430584,25111004,2016-12-08,2016-12-08,True,5,False
5625378,1430584,6606855,2017-03-01,2017-03-01,True,3,False
5625377,1430584,18812405,2017-05-05,2017-05-31,True,3,True
5625376,1430584,18692431,2017-08-02,2017-08-09,True,3,True
